# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [56]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [57]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,el bauga,18.2620,33.9081,32.96,12,26,6.77,SD,1687905362
1,1,port-aux-francais,-49.3500,70.2167,2.60,80,93,8.97,TF,1687905135
2,2,blackmans bay,-43.0167,147.3167,6.07,80,59,4.25,AU,1687905363
3,3,machico,32.7000,-16.7667,28.40,24,0,6.17,PT,1687905363
4,4,adamstown,-25.0660,-130.1015,21.06,74,89,12.15,PN,1687905137


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [58]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng","Lat", geo = True, tiles = "OSM", color = "City",size = "Humidity")

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [60]:
# Narrow down cities that fit criteria and drop any results with null values
city_vacation_df = city_data_df.loc[(city_data_df['Max Temp'] >= 21) & 
                                    (city_data_df['Max Temp'] <= 27) &
                                    (city_data_df['Wind Speed'] < 4.5) &
                                    (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
city_vacation_df.dropna

# Display sample data
city_vacation_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,macae,-22.3708,-41.7869,22.86,83,0,3.09,BR,1687905367
148,148,san lawrenz,36.0553,14.2040,23.25,89,0,2.56,MT,1687905390
247,247,lata,40.1629,-8.3327,23.29,83,0,0.60,PT,1687905220
283,283,paraty,-23.2178,-44.7131,21.12,59,0,1.67,BR,1687905411
287,287,placido de castro,-10.3353,-67.1856,25.91,57,0,1.09,BR,1687905411
337,337,kayabaglar,37.9825,41.6653,21.29,26,0,0.85,TR,1687905419
339,339,westport,41.1415,-73.3579,25.71,78,0,4.12,US,1687905258
381,381,trani,41.2727,16.4154,23.81,59,0,1.13,IT,1687905428
452,452,rock falls,41.7798,-89.6890,26.99,52,0,2.06,US,1687905438
461,461,forio,40.7342,13.8646,24.18,90,0,0.45,IT,1687905439


### Step 3: Create a new DataFrame called `hotel_df`.

In [61]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_vacation_df.copy()
hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
25,macae,BR,-22.3708,-41.7869,83,
148,san lawrenz,MT,36.0553,14.2040,89,
247,lata,PT,40.1629,-8.3327,83,
283,paraty,BR,-23.2178,-44.7131,59,
287,placido de castro,BR,-10.3353,-67.1856,57,
337,kayabaglar,TR,37.9825,41.6653,26,
339,westport,US,41.1415,-73.3579,78,
381,trani,IT,41.2727,16.4154,59,
452,rock falls,US,41.7798,-89.6890,52,
461,forio,IT,40.7342,13.8646,90,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [62]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":100
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
macae - nearest hotel: Hotel Plaza
san lawrenz - nearest hotel: Ta Majsi
lata - nearest hotel: Residencial Botânico
paraty - nearest hotel: Solar dos Gerânios
placido de castro - nearest hotel: No hotel found
kayabaglar - nearest hotel: No hotel found
westport - nearest hotel: Norwalk Inn and Conference Center
trani - nearest hotel: Hotel Trani
rock falls - nearest hotel: Holiday Inn Express & Suites
forio - nearest hotel: Hotel Terme Colella
darwin - nearest hotel: Mantra Pantanas Darwin
vila velha - nearest hotel: Hotel Prainha
alexandria - nearest hotel: Smoha Zahran Haus (7th floor)
quillacollo - nearest hotel: Casa Campestre
tazacorte - nearest hotel: App Leyma
remire-montjoly - nearest hotel: Complexe Belova


,City,Country,Lat,Lng,Humidity,Hotel Name
25,macae,BR,-22.3708,-41.7869,83,Hotel Plaza
148,san lawrenz,MT,36.0553,14.2040,89,Ta Majsi
247,lata,PT,40.1629,-8.3327,83,Residencial Botânico
283,paraty,BR,-23.2178,-44.7131,59,Solar dos Gerânios
287,placido de castro,BR,-10.3353,-67.1856,57,No hotel found
337,kayabaglar,TR,37.9825,41.6653,26,No hotel found
339,westport,US,41.1415,-73.3579,78,Norwalk Inn and Conference Center
381,trani,IT,41.2727,16.4154,59,Hotel Trani
452,rock falls,US,41.7798,-89.6890,52,Holiday Inn Express & Suites
461,forio,IT,40.7342,13.8646,90,Hotel Terme Colella


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [65]:
%%capture --no-display

# Configure the map plot

map_plot2 = hotel_df.hvplot.points("Lng","Lat", frame_width= 700, frame_height= 500, 
geo = True, tiles = "OSM", color = "City", size = "Humidity", hover_cols= ["Country","Hotel Name"])

# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)